`pidgin.weave` controls second of the literate computing REPL.  It controls how the source code is display.  

>>> assert pidgin.weave.__f

In [1]:
    if __name__ == '__main__':
        %reload_ext pidgin

`pidgin.weave` establishes __Markdown__ rules for displaying cell inputs after the `ip.run_cell` has been evaluated.

`pidgin.weave` is a secondary literate program with rules for displaying input encoded in `mistune.markdown`.

In [2]:
    import re, dataclasses, ast, re,traceback, functools, IPython, jinja2, inspect, slugify, collections, html, fnmatch, mistune, inspect, importlib, types, base64, string, traitlets, yaml, io, textwrap
    import doctest, pygments
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    ip = IPython.get_ipython()

    with __import__('importnb').Notebook():
        try:  from . import templating, doctesting, tangle, code_formatting
        except:   import templating, doctesting, tangle, code_formatting

In [3]:
    class PidginWeave_Tangle(traitlets.config.SingletonConfigurable):
The `PidginWeave_Tangle` is a second execution of the source with the intent of display.  
        
        inline = traitlets.Bool(
            True, help=
Evaluate inline statements and expressions
        
        )
`PidginWeave_Tangle.inline` originally only allowed `ip.user_expressions`, but that was too limiting.  Inline statements and 
`doctest`s will both update state.  This creates two separate namespaces the Module namespace rfom the tangle step and the 
woven display code.

        doctest = traitlets.Bool(True, help=
Discover `doctest`s in the source and evaluate them.            
            
        ); format_titles = traitlets.Bool(True, help=
Format code with inspection overlays about the Python objects.            
            
        )
        highlight = None
> `PidginWeave_Tangle.highlight` will use `pygments` to supply extra formatting.  This is unpleasant on rise so we ignore it at the moment.

'    class PidginWeave_Tangle(traitlets.config.SingletonConfigurable):\nThe `PidginWeave_Tangle` is a second execution of the source with the intent of display.  \n        \n        inline = traitlets.Bool(\n            True, help=\nEvaluate inline statements and expressions\n        \n        )\n`PidginWeave_Tangle.inline` originally only allowed `ip.user_expressions`, but that was too limiting.  Inline statements and \n`doctest`s will both update state.  This creates two separate namespaces the Module namespace rfom the tangle step and the \nwoven display code.\n\n        doctest = traitlets.Bool(True, help=\nDiscover `doctest`s in the source and evaluate them.            \n            \n        ); format_titles = traitlets.Bool(True, help=\nFormat code with inspection overlays about the Python objects.            \n            \n        )\n        highlight = None\n> `PidginWeave_Tangle.highlight` will use `pygments` to supply extra formatting.  This is unpleasant on rise so we ignore it at the moment.'

In [4]:
    config_tangle = PidginWeave_Tangle().instance()

In [5]:
    class PidginWeave(traitlets.config.SingletonConfigurable): 
`PidginWeave` modifies the __weave__ phase of the cell execution.

        front_matter = traitlets.Bool(True, help=
Parse `yaml` front matter into the `jinja2` template.
                                      
        ); highlight_expections = traitlets.Bool(True, help=
Highlight code objects that have failed.
                                             
        ); template = traitlets.Bool(True, help=
Template __Markdown__ with `jinja2` before parsing.
                                             
        ); safe = traitlets.Bool(True, help=
Replace file paths with testable and safe strings. 
                                 
        )
    
        tangle = config_tangle
        
> safe mode is implemented in a suboptimal way at the moment.

'    class PidginWeave(traitlets.config.SingletonConfigurable): \n`PidginWeave` modifies the __weave__ phase of the cell execution.\n\n        front_matter = traitlets.Bool(True, help=\nParse `yaml` front matter into the `jinja2` template.\n                                      \n        ); highlight_expections = traitlets.Bool(True, help=\nHighlight code objects that have failed.\n                                             \n        ); template = traitlets.Bool(True, help=\nTemplate __Markdown__ with `jinja2` before parsing.\n                                             \n        ); safe = traitlets.Bool(True, help=\nReplace file paths with testable and safe strings. \n                                 \n        )\n    \n        tangle = config_tangle\n        \n> safe mode is implemented in a suboptimal way at the moment.'

In [6]:
    config = PidginWeave().instance()

> The following regular expressions are copied from `mistune`.

`BlockGrammar` includes doctype and prolog parsing in the html and should be used with `BlockLexer`.

`InlineGrammar` includes doctype and prolog parsing in the html and should be used with `InlineLexer`.

In [7]:
    class PidginInlineGrammar(mistune.InlineGrammar):
        """Add svg constructs to inline_html regex."""
        inline_html = re.compile(
                r'^(?:%s|%s|%s)' % (
                    r'<!--[\s\S]*?-->|<!DOCTYPE [\s\S]*?>|<\?[\s\S]*?\?>',
                    r'<(\w+%s)((?:%s)*?)\s*>([\s\S]*?)<\/\1>' % (
                        mistune._valid_end, mistune._valid_attr),
                    r'<\w+%s(?:%s)*?\s*\/?>' % (mistune._valid_end, mistune._valid_attr)))

In [8]:
    if __name__ == '__main__':ip  =get_ipython()

In [9]:
    error = "background-color: var(--jp-rendermime-error-background);"
    ws = "white-space: pre;"

In [10]:
    class WeavePidginBlockLexer(tangle.PidginBlockLexer):                
        def parse(self, text, rules=None):
            if rules == mistune.BlockLexer.footnote_rules:
                text = templating.finalize_ipython(bundle=IPython.get_ipython().display_formatter.format(text.rstrip())[0])
            return super().parse(text)
        
        def parse_doctest(self, m):
            text = ''.join(self.to_string(m))
            ip = IPython.get_ipython()
            if ip and ip.weave.tangle.doctest:
                with IPython.utils.capture.capture_output() as out:
                    runner = doctesting.run_docstring_examples(text, IPython.get_ipython(), verbose=True)
                if runner.failures:
                    IPython.get_ipython().showtraceback((doctesting.DocTestException, doctesting.DocTestException(out.stdout), BaseException().__traceback__))
                    self.failures.update({text: out.stdout})
                else:
                    self.successes[text] = out.stdout
            return super().parse_doctest(m)
        
        def parse_heading(self, m):
            super().parse_heading(m)
            self.tokens[-1]['id']  = slugify.slugify(self.tokens[-1]['text'] )

In [11]:
    class WeavePidginInlineLexer(mistune.InlineLexer):
        """WeavePidginInlineLexer is responsible for updating modifying the interactive IPython state.
        """
        grammar_class = PidginInlineGrammar
        tmp_code = []
                
        def output_code(self, m):
            """Execute inline code."""
            ip = IPython.get_ipython()
            text = '\n'.join(self.tmp_code + [m.group(2)])
            # Code can exist across multiple cells.
            if text.endswith('\\'): self.tmp_code = text.splitlines()
            elif ip and ip.weave.tangle.inline:
                # Evaluate inline code.
                with IPython.utils.capture.capture_output() as out:
                    
                    result = IPython.get_ipython().run_cell(textwrap.indent(text, ' '*4), store_history=False, silent=True)
                if result.error_in_exec or result.error_before_exec:
                    self.renderer.failures[text] = str(
                        result.error_in_exec or result.error_before_exec)                

            return super().output_code(m)

        def output_reflink(self, m):
            """Format the reference link."""
            ip = IPython.get_ipython()
            if not ip: return super().output_reflink(m)
            user_ns = ip.user_ns
            key = mistune._keyify(m.group(2) or m.group(1))
            ret = {'link': '#', 'title': ''}
            if key in self.links: ret.update(self.links[key])
            if key in user_ns:
                try: ret.update({'title': code_formatting.format_title(ip.object_inspect_text(key))})
                except: ...

            elif key not in self.links:
                return 
            return self._process_link(m, ret['link'], ret['title'])
        
        def _process_link(self, m=None, link=None, title=None):
            """Add {link} to the global namespace for easier access."""
            for str in [link]+([''.join(tangle.PidginBlockLexer.to_string(m))] if m else []):
                try: 
                    ast.parse(str)
                except: 
                    IPython.get_ipython().user_ns.update({str:str})
            if m: return super()._process_link(m, link, title)
            
        def output_url(self, m):
            self._process_link(link=m.group(1))
            return super().output_url(m)
        
        def output_footnote(self, m):
            key = mistune._keyify(m.group(1))
            if key in self.footnotes:
                self.footnote_index += 1
                self.footnotes[key] = self.footnote_index
            return self.renderer.footnote_ref(key, self.footnote_index)

In [12]:
    class WeaveRenderer(mistune.Renderer):
        """WeaveRenderer is responsbile for formatting the output of the input code."""
        failures = None
        failure = None
        def block_code(self, text, lang=None):
            style = ws
            if lang is None: 
                if self.failure: style += error
                else: 
                    text = code_formatting.markup_code(text)
                return F"""<code style="{style}" title="{self.failure or ""}">{text}</code>"""
            return super().block_code(text, lang)
        
        def codespan(self, text):
            if text in (self.failures or {}):
                return F"""<code style="{error}" title="{code_formatting.format_title(
                    self.failures.pop(text)
                )}">{text}</code>"""
            return F"""<code>{code_formatting.markup_code_inline(text)}</code>"""
        
        def text(self, text): return text
        
        def doctest(self, text):
            if text in (self.failures or {}):
                return F"""<code style="{ws + error}" title="{code_formatting.format_title(self.failures.pop(text))
                }">{text}</code>"""
            if text in (self.successes or {}):
                return F"""<code style="{ws}" title="{code_formatting.format_title(self.successes.pop(text))
                }">{code_formatting.markup_code(text)}</code>"""  
            return ""

        
        def footnote_ref(self, key, index):
            return F"""<sup class="footnote-ref" id="fnref-{slugify.slugify(key)}">
                <a title="{code_formatting.format_title(IPython.get_ipython().user_ns.get(key, ""))
                }" href="#fn-{mistune.escape(key)}">{key}</a></sup>"""
        
        def header(self, text, level, raw=None):
            id = slugify.slugify(raw)            
            link = '<a href="#%s">{}</a>\n' % id
            IPython.get_ipython().user_ns[raw] = IPython.display.HTML(link.format(text))
            return link.format('<h%d id="%s">%s</h%d>\n'% (level, id, text, level)) 

In [13]:
    class WeavePidginMarkdown(mistune.Markdown):
        """WeavePidginMarkdown combines initializes and applies WeavePidginBlockLexer and WeavePidginInlineLexer.
        """
        def output_doctest(self):  return self.renderer.doctest(self.token['text'])
        def pop(self):
            if not self.tokens: return None
            self.token = self.tokens.pop()
            return self.token
        
        def render(self, text, failure=None):
            self.renderer.failure = failure
            return super().render(text)
        
        def parse(self, text):
            self.block.failures = self.renderer.failures = {}
            self.block.successes = self.renderer.successes = {}
            self.inline.tmp_code = []
            out = self.output(mistune.preprocessing(text))
            keys = self.block.def_footnotes
            self.block.def_footnotes = {}
            self.inline.footnotes = {}
            if not self.footnotes: return out

            self.footnotes = sorted(self.footnotes, key=lambda o: keys.get(o['key']), reverse=True)

            body = self.renderer.placeholder()
            while self.footnotes:
                note = self.footnotes.pop()
                body += self.renderer.footnote_item(note['key'], note['text'])

            out += self.renderer.footnotes(body)
            return out

In [14]:
    render = WeavePidginMarkdown(
        renderer=WeaveRenderer(escape=False, parse_block_html=True, parse_inline_html=False),
        block=WeavePidginBlockLexer, inline=WeavePidginInlineLexer)

In [15]:
    def front_matter(str):
        if str.startswith('---'):
            if '\n---' in str:
                body = str.lstrip('---')
                data, sep, body = body.partition('---\n')
                data = yaml.safe_load(io.StringIO(data))
                if isinstance(data, dict): return body, data
        return str, {}

In [16]:
    @dataclasses.dataclass(order=True)
    class PidginDisplay:
        data: str = ''
        ip: None = dataclasses.field(default_factory=IPython.get_ipython, repr=False)
        ns: None = dataclasses.field(default_factory=dict, repr=False)
        def __post_init__(self):
            if self.ip and not self.ns: self.ns = self.ip.user_ns
        
        def _repr_mimebundle_(self, include=None, exclude=None):
            env = getattr(self.ip, 'environment', templating.new_environment(self.ip))
            exc = None
            data = {}
            if self.ip.weave.front_matter:
                string, data = front_matter(self.data)
            lines = string.splitlines(True)    
            lineno = len(lines)

            if getattr(self.ip, '_last_traceback', None) and self.ip.weave.highlight_expections: 
                exc = self.ip._get_exc_info()
                if issubclass(exc[0], SyntaxError): lineno = exc[1].args[1][1] - 1
                else: lineno = traceback.extract_tb(exc[-1])[-1].lineno - 1
            
            template =  bool(all(
                map(self.data.__contains__, {env.block_start_string, env.block_end_string})
            ) or all(
                map(self.data.__contains__, {env.variable_start_string, env.variable_end_string})
            ))
            body = """"""
            for i, range in enumerate((slice(lineno), slice(lineno, None))):
                blob = ''.join(lines[range])
                if template and self.ip.weave.template:
                    blob = env.from_string(blob).render(self.ns, **data)
                body += render.render(blob, failure=str(exc[1]) if (i == 1) and exc else None)
                
            # This failures when templates are broken across errors
                
            return {'text/html': body,  'text/markdown': self.data, 'text/plain': self.data}, {}

`run_cell_display` triggers a display on the input if contains anything other than block code.

In [17]:
    def run_cell_display(result) -> bool:
        raw, source = result.info.raw_cell, tangle.markdown_to_python(result.info.raw_cell)
        try:
            "Blank first line"
            is_display = bool(raw.partition('\n')[0].strip() and not raw.startswith(';')) 
            if raw.strip() == ast.literal_eval(source.rstrip().rstrip(';')).strip():
                is_display = False
        except: ...
        
        if (result.error_before_exec or result.error_in_exec) or (is_display and not only_contains_block_code(raw, source)):
            IPython.display.display(raw)

`only_contains_block_code` determines if the input source and python only include block code.

In [18]:
    def only_contains_block_code(raw: str, source: str) -> bool: 
        return list(
            map(str.rstrip, textwrap.dedent(raw).splitlines())
        ) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))

In [19]:
    def load_ipython_extension(ip): 
>>> load_ipython_extension(get_ipython())        

        ip.weave = config
        ip.events.register('post_run_cell', run_cell_display)

    def unload_ipython_extension(ip=None): 
>>> unload_ipython_extension(get_ipython())                

        ip.events.callbacks['post_run_cell'] = [
            object for object in ip.events.callbacks['post_run_cell']
            if object is not run_cell_display
        ]

"    def load_ipython_extension(ip): \n>>> load_ipython_extension(get_ipython())        \n\n        ip.weave = config\n        ip.events.register('post_run_cell', run_cell_display)\n\n    def unload_ipython_extension(ip=None): \n>>> unload_ipython_extension(get_ipython())                \n\n        ip.events.callbacks['post_run_cell'] = [\n            object for object in ip.events.callbacks['post_run_cell']\n            if object is not run_cell_display\n        ]"